<a href="https://colab.research.google.com/github/rita-sasaki/2024slowEQ/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#辞書型で変数セットする場合


#辞書型よりもデータフレーム型の方が微分計算など良さそう
variables = {col: data[col] for col in data.columns} # 各列を各変数として定義（辞書型変数を使用，dataframe型と違って列の順序は無関係）


# 最初数行を表示
print(data.head())


# 必要なデータ列の計算・追加
variables['angv'] = [(360/60)*safe_float(rpm_str.strip("[]").split('=')[1]) for rpm_str in data['RPM']] # deg/s
variables['tor'] = [(0.0673*(10**(-3))/100)*safe_float(tor_str.strip("[]").split('=')[1]) for tor_str in data['TOR']] # N m
variables['spindle'] = safe_float(data['SPINDLE'][0].strip("[]").split('=')[1]) # 使用スピンドル記号,数字一つだけのデータ（配列ではない）
#variables['angv'] = [(360/60)*safe_float(rpm_str.strip("[]").split('=')[1]) for rpm_str in data['RPM']] # 使用装置
#variables['vis'] = [0.001*safe_float(vis_str.strip("[]").split('=')[1]) for vis_str in data['VIS']] # Pa s
#variables['strs'] = [safe_float(strs_str.strip("[]").split('=')[1]) for strs_str in data['STRS']] # Pa
#variables['strnr'] = [safe_float(strnr_str.strip("[]").split('=')[1]) for strnr_str in data['STRNR']] # /s
variables['tem'] = [safe_float(tem_str.strip("[C]").split('=')[1]) for tem_str in data['TEM']] # degreeC

# 時刻をUNIX時間に変更
date_str = [datetime_str.strip("[]") for datetime_str in data['Y/D'] + " " + data['HMS']] # リストとして一括計算するときは[]で囲む
timestamps = [dt.strptime(date, "%Y-%m-%d %H:%M:%S.%f").timestamp() for date in date_str] # 文字列date_strをstrptimeでdatetime型に変換して，さらにtimestampでunix時間（UTC時刻1970年1月1日午前0時0分0秒（UNIXエポック）からの経過秒数）に変換する
# 変形時間
first_nonzero_rpm_index = next(i for i, x in enumerate(variables['angv']) if x != 0) # rpm=0が初めて0でなくなった要素のインデックスを取得（enumerate(x)はリストxのインデックスi・要素xの組をイテレータ（連番ポインタ）として出力し，そのうちxの条件に次に（最初に）合致するiをnextが出力）
variables['time'] = [x - timestamps[first_nonzero_rpm_index] for x in timestamps]


#　トルクデータから変位を推定
if variables['spindle'] == 61: #　スピンドル番号に応じたスピンドル直径
    dia = 18.84
elif variables['spindle'] == 62:
    dia = 18.72
else:
    pass
# mm
variables['disp'] = [((angv * time * 0.0000009753) - tor) / (360*0.0000009753 / (dia * np.pi))
                    for angv, time, tor in zip(variables['angv'], variables['time'], variables['tor'])] # 複数リストの要素をイテラブルの連番ポインタとして引数に取るにはzipで囲って並列で示す必要がある
# トータル回転角度の計算 angv*time は　angv一定を前提としているので，実験途中に回転速度を変えた場合は要注意